In [1]:
from model import GPT
import numpy as np
import torch
from tqdm import tqdm
from copy import deepcopy

In [2]:
model = GPT.from_pretrained('/data2/mengfanxu/huggingface/gpt2-xl')
orthogonal_model = deepcopy(model)
state_dict = torch.load("orthogonal/gpt2-xl.pt")
orthogonal_model.load_state_dict(state_dict)

/home/mengfanxu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 1555.97M


/tmp/ipykernel_3186120/3626383437.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("orthogonal/gpt2-xl.pt")


<All keys matched successfully>

In [9]:
orthogonal_model.config.n_layer,orthogonal_model.config.n_embd

(48, 1600)

In [16]:
qk_norm_list = []
vo_norm_list = []
for name, module in orthogonal_model.named_modules():
    if name.endswith("attn"):
        q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
        q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
        
        k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
        k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
        qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
        qk_norm_list.append(qk_norm)

In [17]:
qk_norm_list = torch.stack(qk_norm_list)

In [22]:
for norm in qk_norm_list:
    print(norm)

tensor([75.6407, 59.7586, 34.3032,  ...,  0.7776,  0.7452,  0.6627])
tensor([24.5278, 14.9114, 11.4587,  ...,  0.6109,  0.5813,  0.5491])
tensor([33.4253, 19.3119, 12.8235,  ...,  0.7925,  0.7374,  0.6017])
tensor([12.3362, 10.8317,  9.7711,  ...,  1.1934,  0.8758,  0.8080])
tensor([6.9461, 5.2508, 4.8550,  ..., 1.2732, 0.8975, 0.8220])
tensor([8.8811, 7.5086, 6.9884,  ..., 1.4820, 0.9754, 0.7467])
tensor([8.8682, 6.5936, 5.0357,  ..., 1.4805, 1.4434, 1.4173])
tensor([11.4886,  8.5152,  8.1921,  ...,  2.0938,  2.0079,  1.8927])
tensor([9.0519, 8.7206, 7.3931,  ..., 2.2672, 0.7412, 0.7057])
tensor([11.4003, 10.5980,  7.0392,  ...,  1.6157,  1.5513,  1.4216])
tensor([10.2905, 10.0548,  7.5150,  ...,  2.3830,  2.2690,  2.2384])
tensor([8.2963, 6.5400, 5.9895,  ..., 2.3059, 2.1929, 2.0998])
tensor([7.9127, 7.0393, 6.8282,  ..., 2.7395, 2.6665, 2.6385])
tensor([6.8908, 5.9467, 5.7999,  ..., 1.8580, 1.8460, 1.5443])
tensor([6.9871, 5.6310, 5.0021,  ..., 1.9697, 1.8825, 1.8153])
tensor([7.971

In [27]:
from datasets import load_from_disk
dataset = load_from_disk("wikitext-2-raw-v1")

In [29]:
dataset['test']

Dataset({
    features: ['text'],
    num_rows: 4358
})

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from accelerate.test_utils.testing import get_backend

device = 'cpu'
model_id = "/data2/mengfanxu/huggingface/gpt2-xl"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

from datasets import load_from_disk
test = load_from_disk("wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nll_sum = 0.0
n_tokens = 0
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    # Accumulate the total negative log-likelihood and the total number of tokens
    num_valid_tokens = (target_ids != -100).sum().item()  # number of valid tokens in target_ids
    batch_size = target_ids.size(0)
    num_loss_tokens = num_valid_tokens - batch_size  # subtract batch_size due to internal label shift
    nll_sum += neg_log_likelihood * num_loss_tokens
    n_tokens += num_loss_tokens

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

avg_nll = nll_sum / n_tokens  # average negative log-likelihood per token
ppl = torch.exp(avg_nll)

OSError: Can't load tokenizer for '/data2/mengfanxu/huggingface/gpt2-xl'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/data2/mengfanxu/huggingface/gpt2-xl' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.